In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import optimize, interpolate

class interp_LHD_data():
    def __init__(self, *args, **kwargs):
        file_p = kwargs.get("file_p")
        file_q = kwargs.get("file_q")        
        data_p = np.loadtxt(file_p,skiprows=3)
        data_q = np.loadtxt(file_q,skiprows=3)
        with open(file_p,mode="r") as f:
            line = f.readlines()[2]
            label = line.split()
        # print(label,data_p.shape, data_q.shape)
        p_interp = interpolate.CubicSpline(data_p[:,0],data_p[:,1:],axis=0)
        q_interp = interpolate.CubicSpline(data_q[:,0],data_q[:,1])
        self.__label = label
        self.__p_interp = p_interp
        self.__q_interp = q_interp
        return
    
    def get_profile(self, rho):
        plasma = self.__p_interp(rho)
        q_0 = self.__q_interp(rho)        
        dqdrho = self.__q_interp.derivative()(rho)
        s_hat = rho * dqdrho / q_0
        prof = np.concatenate([rho[:,np.newaxis],plasma,q_0[:,np.newaxis],s_hat[:,np.newaxis]],axis=1)
        label = self.__label.copy()
        label.append("q_0")
        label.append("s_hat")
        return prof, label

def read_LHD_model(file_m):
    data = np.loadtxt(file_m,skiprows=3)
    with open(file_m,mode="r") as f:
        line = f.readlines()[2]
        label = line.split()
    return data, label

interpLHDplasma1 = interp_LHD_data(file_p="LHDplasma1.txt",file_q="LHDq1.txt")
interpLHDplasma2 = interp_LHD_data(file_p="LHDplasma2.txt",file_q="LHDq2.txt")
interpLHDplasma3 = interp_LHD_data(file_p="LHDplasma3.txt",file_q="LHDq3.txt")

model1, label_m1 = read_LHD_model(file_m="LHDmodel1.txt")
model2, label_m2 = read_LHD_model(file_m="LHDmodel2.txt")
model3, label_m3 = read_LHD_model(file_m="LHDmodel3.txt")

prof1, label_p1 = interpLHDplasma1.get_profile(model1[:,0])
prof2, label_p2 = interpLHDplasma2.get_profile(model2[:,0])
prof3, label_p3 = interpLHDplasma3.get_profile(model3[:,0])

label1 = label_p1 + label_m1[1:]
data1 = np.concatenate([prof1[:,:],model1[:,1:]],axis=1)
label2 = label_p2 + label_m2[1:]
data2 = np.concatenate([prof2[:,:],model2[:,1:]],axis=1)
label3 = label_p3 + label_m3[1:]
data3 = np.concatenate([prof3[:,:],model3[:,1:]],axis=1)

print(len(label1),data1.shape); print(label1)
print(len(label2),data2.shape); print(label2)
print(len(label3),data3.shape); print(label3)

27 (10, 27)
['rho', 'n(10^19m-3)', 'dn/dr', 'Ln', 'R/Ln', 'Te(keV)', 'dTe/dr', 'LTe', 'R/LTe', 'Ti(kev)', 'dTi/dr', 'LTi', 'R/LTi', 'q_0', 's_hat', '\\gamma/ky^2', 'zfdt', '\\chi_e_nlnr', '\\chi_e_model', 'Qe_nlnr', 'Qe_qslmodel', '\\chi_i_nlnr', '\\chi_i_model', 'Qi_nlnr', 'Qi_qslmodel', '\\Gamma_nlnr', '\\Gamma_qslmodel']
21 (5, 21)
['rho', 'n(10^19m-3)', '(dn/dr)^-1', 'Ln', 'R/Ln', 'Te=Ti(keV)', 'R/LTi', 'q_0', 's_hat', '\\gamma/ky^2', 'zfdt', '\\chi_e_nlnr', '\\chi_e_model', 'Qe_nlnr', 'Qe_qslmodel', '\\chi_i_nlnr', '\\chi_i_model', 'Qi_nlnr', 'Qi_qslmodel', '\\Gamma_nlnr', '\\Gamma_qslmodel']
22 (5, 22)
['rho', 'n(10^19m-3)', '(dn/dr)', 'Ln', 'R/Ln', 'Te=Ti(keV)', 'dTi/dr', 'R/LTi', 'q_0', 's_hat', '\\gamma/ky^2', 'zfdt', '\\chi_e_nlnr', '\\chi_e_model', 'Qe_nlnr', 'Qe_qslmodel', '\\chi_i_nlnr', '\\chi_i_model', 'Qi_nlnr', 'Qi_qslmodel', '\\Gamma_nlnr', '\\Gamma_qslmodel']


In [2]:
dataset1 = np.concatenate([data1[:,0:1],  # rho
                           data1[:,1:2],  # n(10^19m-3)
                           data1[:,4:5],  # R/Ln
                           data1[:,5:6],  # Te(keV)
                           data1[:,8:9],  # R/LTe
                           data1[:,9:10],  # Ti(keV)
                           data1[:,12:13], # R/LTi
                           data1[:,13:14], # q_0
                           data1[:,14:15], # s_hat
                           data1[:,15:16], # gamma/ky^2
                           data1[:,16:17], # zfdt
                           data1[:,20:21], # Qe_qslmodel
                           data1[:,24:25], # Qi_qslmodel
                           data1[:,26:27], # Gamma_qslmodel
                           data1[:,19:20], # Qe_nlnr
                           data1[:,23:24], # Qi_nlnr
                           data1[:,25:26]  # Gamma_nlnr
                          ], axis=1)
print(dataset1.shape)

(10, 17)


In [3]:
dataset2 = np.concatenate([data2[:,0:1],  # rho
                           data2[:,1:2],  # n(10^19m-3)
                           data2[:,4:5],  # R/Ln
                           data2[:,5:6],  # Te(keV)
                           data2[:,6:7],  # R/LTe
                           data2[:,5:6],  # Ti(keV)
                           data2[:,6:7],  # R/LTi
                           data2[:,7:8],  # q_0
                           data2[:,8:9],  # s_hat
                           data2[:,9:10], # gamma/ky^2
                           data2[:,10:11], # zfdt
                           data2[:,14:15], # Qe_qslmodel
                           data2[:,18:19], # Qi_qslmodel
                           data2[:,20:21], # Gamma_qslmodel
                           data2[:,13:14], # Qe_nlnr
                           data2[:,17:18], # Qi_nlnr
                           data2[:,19:20]  # Gamma_nlnr
                          ], axis=1)
print(dataset2.shape)

(5, 17)


In [4]:
dataset3 = np.concatenate([data3[:,0:1],  # rho
                           data3[:,1:2],  # n(10^19m-3)
                           data3[:,4:5],  # R/Ln
                           data3[:,5:6],  # Te(keV)
                           data3[:,7:8],  # R/LTe
                           data3[:,5:6],  # Ti(keV)
                           data3[:,7:8],  # R/LTi
                           data3[:,8:9],  # q_0
                           data3[:,9:10], # s_hat
                           data3[:,10:11], # gamma/ky^2
                           data3[:,11:12], # zfdt
                           data3[:,15:16], # Qe_qslmodel
                           data3[:,19:20], # Qi_qslmodel
                           data3[:,21:22], # Gamma_qslmodel
                           data3[:,14:15], # Qe_nlnr
                           data3[:,18:19], # Qi_nlnr
                           data3[:,20:21]  # Gamma_nlnr
                          ], axis=1)
print(dataset3.shape)

(5, 17)


In [5]:
label = ["rho", "n(10^19m-3)", "R/Ln", "Te(keV)", "R/LTe", "Ti(keV)", "R/LTi", "q_0", "s_hat",
         "gamma/ky^2", "zfdt", "Qe_qslmodel", "Qi_qslmodel", "Gamma_qslmodel", 
         "Qe_nlnr", "Qi_nlnr", "Gamma_nlnr"]
print(len(label))
print(label)

17
['rho', 'n(10^19m-3)', 'R/Ln', 'Te(keV)', 'R/LTe', 'Ti(keV)', 'R/LTi', 'q_0', 's_hat', 'gamma/ky^2', 'zfdt', 'Qe_qslmodel', 'Qi_qslmodel', 'Gamma_qslmodel', 'Qe_nlnr', 'Qi_nlnr', 'Gamma_nlnr']


In [6]:
df1 = pd.DataFrame(dataset1,columns=label)
df1.to_csv("LHD_df1.csv",index=False)
df2 = pd.DataFrame(dataset2,columns=label)
df2.to_csv("LHD_df2.csv",index=False)
df3 = pd.DataFrame(dataset3,columns=label)
df3.to_csv("LHD_df3.csv",index=False)

In [7]:
print(df1)
print(df2)
print(df3)

    rho  n(10^19m-3)     R/Ln  Te(keV)    R/LTe  Ti(keV)    R/LTi     q_0  \
0  0.46       1.2436 -0.51928   2.9416   4.5804   2.9230   7.9322  2.1956   
1  0.50       1.2486 -0.72958   2.8502   5.3065   2.7717   8.7514  2.0927   
2  0.54       1.2551 -0.90278   2.7479   6.1269   2.6143   9.6056  1.9858   
3  0.58       1.2628 -1.00630   2.6346   7.0567   2.4521  10.4980  1.8764   
4  0.62       1.2710 -1.00200   2.5099   8.1142   2.2867  11.4330  1.7655   
5  0.65       1.2769 -0.90183   2.4090   9.0047   2.1613  12.1640  1.6820   
6  0.68       1.2818 -0.69537   2.3017   9.9917   2.0356  12.9220  1.5986   
7  0.72       1.2857 -0.21250   2.1491  11.4830   1.8684  13.9760  1.4879   
8  0.76       1.2845  0.57186   1.9860  13.2160   1.7033  15.0810  1.3784   
9  0.80       1.2753  1.74850   1.8135  15.2480   1.5416  16.2350  1.2703   

      s_hat  gamma/ky^2  zfdt  Qe_qslmodel  Qi_qslmodel  Gamma_qslmodel  \
0 -0.527293         0.8   2.6         21.0         70.0          4.1000   
1 